In [1]:
from datetime import date
from glob import glob
import pickle

import pandas as pd

In [2]:
!ls -d machine_studies/*/

machine_studies/211125_all_agents_000_002/
machine_studies/211125_polished_donkey_22_66/
machine_studies/211202_all_agents_000_002/
machine_studies/211202_all_agents_000_002_solenoid_adjusted/
machine_studies/211202_polar_lake_000_069/
machine_studies/211202_polished_donkey_003_021/
machine_studies/211202_polished_donkey_067_099/
machine_studies/211209_all_agents_000_002_drift/
machine_studies/211209_hannes_vs_agent_01/
machine_studies/211209_still_deluge_000_050/
machine_studies/211209_still_deluge_050_100/
machine_studies/211209_willi_vs_agent_01/
machine_studies/deluge_100_150/
machine_studies/deluge_150_200/
machine_studies/deluge_200_250/
machine_studies/donkey_100_150/
machine_studies/donkey_150_200/
machine_studies/donkey_200_250/
machine_studies/lake_100_150/
machine_studies/lake_150_200/
machine_studies/lake_200_250/
machine_studies/lake_70_100/
machine_studies/longrun/
machine_studies/shortrun/
machine_studies/snapshots/


## Over Problems Experiments

In [3]:
# Polished Donkey
filepaths = sorted(glob("machine_studies/*donkey*/*.pkl") + glob("machine_studies/211125_all_agents_000_002/*donkey*.pkl"))
dfdonkey = pd.concat([pd.read_pickle(path) for path in filepaths], ignore_index=True)

In [4]:
filepaths = sorted(glob("machine_studies/*lake*/*.pkl"))
dfpolar = pd.concat([pd.read_pickle(path) for path in filepaths], ignore_index=True)

In [5]:
filepaths = sorted(glob("machine_studies/*deluge*/*.pkl"))
dfstill = pd.concat([pd.read_pickle(path) for path in filepaths], ignore_index=True)

In [6]:
df = pd.concat([dfdonkey,dfpolar,dfstill], ignore_index=True)

In [7]:
df.drop("beam_image", axis=1, inplace=True)

In [8]:
df.to_pickle("machine_studies/summary_agentsoverproblems.pkl")

In [9]:
del df
del dfdonkey
del dfpolar
del dfstill

## Drift Experiments

In [10]:
directories = sorted(glob("machine_studies/*_all_agents*"))
directories

['machine_studies/211125_all_agents_000_002',
 'machine_studies/211202_all_agents_000_002',
 'machine_studies/211202_all_agents_000_002_solenoid_adjusted',
 'machine_studies/211209_all_agents_000_002_drift']

In [11]:
comments = [None, None, "solenoid adjusted", None]
assert len(comments) == len(directories)

In [12]:
dates = [
    date(year=2021, month=11, day=25),
    date(year=2021, month=12, day=2),
    date(year=2021, month=12, day=2),
    date(year=2021, month=12, day=9)
]
assert len(dates) == len(directories)

In [13]:
dfs = []
for directory, comment, rundate in zip(directories, comments, dates):
    filepaths = glob(f"{directory}/*.pkl")
    df = pd.concat([pd.read_pickle(path) for path in filepaths], ignore_index=True)
    df["comment"] = comment
    df["date"] = rundate
    dfs.append(df)
df = pd.concat(dfs, ignore_index=True)

In [14]:
df.drop("beam_image", axis=1, inplace=True)

In [15]:
df.to_pickle("machine_studies/summary_driftexperiments.pkl")

In [16]:
del df

## Agents vs. Operators

In [74]:
# Hannes
directory = "machine_studies/211209_hannes_vs_agent_01"
filepaths = glob(f"{directory}/*.pkl")

hannes = []
for path in filepaths:
    with open(path, "rb") as f:
        hannes.append(pickle.load(f))

hannes = {k: v for f in hannes for k, v in f.items()}

dfhannes = pd.DataFrame(hannes)
dfhannes = dfhannes.T
dfhannes = dfhannes.applymap(lambda x: x["data"])

dfhannes = dfhannes[[
    "SINBAD.MAGNETS/MAGNET.ML/AREAMQZM1/STRENGTH.RBV",
    "SINBAD.MAGNETS/MAGNET.ML/AREAMQZM2/STRENGTH.RBV",
    "SINBAD.MAGNETS/MAGNET.ML/AREAMQZM3/STRENGTH.RBV",
    "SINBAD.MAGNETS/MAGNET.ML/AREAMCVM1/KICK_MRAD.RBV",
    "SINBAD.MAGNETS/MAGNET.ML/AREAMCHM1/KICK_MRAD.RBV",
    "SINBAD.DIAG/CAMERA/AR.EA.BSC.R.1/IMAGE_EXT_ZMQ"
]]
dfhannes.rename(columns={
    "SINBAD.MAGNETS/MAGNET.ML/AREAMQZM1/STRENGTH.RBV": "q1",
    "SINBAD.MAGNETS/MAGNET.ML/AREAMQZM2/STRENGTH.RBV": "q2",
    "SINBAD.MAGNETS/MAGNET.ML/AREAMQZM3/STRENGTH.RBV": "q3",
    "SINBAD.MAGNETS/MAGNET.ML/AREAMCVM1/KICK_MRAD.RBV": "cv",
    "SINBAD.MAGNETS/MAGNET.ML/AREAMCHM1/KICK_MRAD.RBV": "ch",
    "SINBAD.DIAG/CAMERA/AR.EA.BSC.R.1/IMAGE_EXT_ZMQ": "beam_image"
}, inplace=True)

dfhannes = dfhannes.reset_index().rename(columns={"index": "step"})
dfhannes["model"] = "Hannes"
dfhannes["method"] = "operator"
dfhannes["description"] = "Beam optimisation performed by human operator"
dfhannes["cv"] = dfhannes["cv"] * 1e-3
dfhannes["ch"] = dfhannes["ch"] * 1e-3
dfhannes["problem"] = 0
dfhannes["mup_x"] = 0.0
dfhannes["mup_y"] = 0.0
dfhannes["sigmap_x"] = 0.0
dfhannes["sigmap_y"] = 0.0

In [75]:
# Willi
directory = "machine_studies/211209_willi_vs_agent_01"
filepaths = glob(f"{directory}/*.pkl")

willi = []
for path in filepaths:
    with open(path, "rb") as f:
        willi.append(pickle.load(f))

willi = {k: v for f in willi for k, v in f.items()}

dfwilli = pd.DataFrame(willi)
dfwilli = dfwilli.T
dfwilli = dfwilli.applymap(lambda x: x["data"])

dfwilli = dfwilli[[
    "SINBAD.MAGNETS/MAGNET.ML/AREAMQZM1/STRENGTH.RBV",
    "SINBAD.MAGNETS/MAGNET.ML/AREAMQZM2/STRENGTH.RBV",
    "SINBAD.MAGNETS/MAGNET.ML/AREAMQZM3/STRENGTH.RBV",
    "SINBAD.MAGNETS/MAGNET.ML/AREAMCVM1/KICK_MRAD.RBV",
    "SINBAD.MAGNETS/MAGNET.ML/AREAMCHM1/KICK_MRAD.RBV",
    "SINBAD.DIAG/CAMERA/AR.EA.BSC.R.1/IMAGE_EXT_ZMQ"
]]
dfwilli.rename(columns={
    "SINBAD.MAGNETS/MAGNET.ML/AREAMQZM1/STRENGTH.RBV": "q1",
    "SINBAD.MAGNETS/MAGNET.ML/AREAMQZM2/STRENGTH.RBV": "q2",
    "SINBAD.MAGNETS/MAGNET.ML/AREAMQZM3/STRENGTH.RBV": "q3",
    "SINBAD.MAGNETS/MAGNET.ML/AREAMCVM1/KICK_MRAD.RBV": "cv",
    "SINBAD.MAGNETS/MAGNET.ML/AREAMCHM1/KICK_MRAD.RBV": "ch",
    "SINBAD.DIAG/CAMERA/AR.EA.BSC.R.1/IMAGE_EXT_ZMQ": "beam_image"
}, inplace=True)

dfwilli = dfwilli.reset_index().rename(columns={"index": "step"})
dfwilli["model"] = "Willi"
dfwilli["method"] = "operator"
dfwilli["description"] = "Beam optimisation performed by human operator"
dfwilli["cv"] = dfwilli["cv"] * 1e-3
dfwilli["ch"] = dfwilli["ch"] * 1e-3
dfwilli["problem"] = 0
dfwilli["mup_x"] = 0.0
dfwilli["mup_y"] = 0.0
dfwilli["sigmap_x"] = 0.0
dfwilli["sigmap_y"] = 0.0

In [76]:
# Agents
filepaths = glob("machine_studies/211209_all_agents_000_002*/*_000_*.pkl")
dfagents = pd.concat([pd.read_pickle(path) for path in filepaths], ignore_index=True)

In [78]:
df = pd.concat([dfhannes,dfwilli,dfagents], ignore_index=True)

In [79]:
df.drop("beam_image", axis=1, inplace=True)

In [81]:
df.to_pickle("machine_studies/summary_operatorsvsagents.pkl")

## Long Run

In [19]:
path = glob("machine_studies/longrun/*")[0]
df = pd.read_pickle(path)

In [21]:
df.drop("beam_image", axis=1, inplace=True)

In [22]:
df.to_pickle("machine_studies/summary_longrun.pkl")

In [23]:
del df

## Upstream Reaction